In [3]:
from pyspark.sql import  SQLContext, Row
from pyspark import SparkContext,SparkConf
import pandas as pd
from pyspark.sql.functions import regexp_extract, regexp_replace, when,udf,col

In [4]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc )

In [5]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]
empA=sqlCtx.createDataFrame( emp, ['name','deptid'] )
deptB=sqlCtx.createDataFrame( dept,['deptname','deptid'] )

In [6]:
empA.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [9]:
deptB.show()

+--------+------+
|deptname|deptid|
+--------+------+
|    개발|     1|
|    연구|     2|
|    영업|     3|
|    기획|     4|
+--------+------+



# join

- 여러 df를 합처준다.
- defalt param은 inner join

In [12]:
empA.join(deptB, on = ['deptid'], how = 'inner').show()

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
+------+------+--------+



In [14]:
empA.join(deptB, on = ['deptid'], how = 'left').show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
+------+-------+--------+



In [15]:
empA.join(deptB, on = ['deptid'], how = 'right').show()

+------+------+--------+
|deptid|  name|deptname|
+------+------+--------+
|     1|홍길동|    개발|
|     3|임꺽정|    영업|
|     3|김철수|    영업|
|     2|이순신|    연구|
|     4|  null|    기획|
+------+------+--------+



In [16]:
empA.join(deptB, on = ['deptid'], how = 'full').show()

+------+-------+--------+
|deptid|   name|deptname|
+------+-------+--------+
|     5|김철수1|    null|
|     1| 홍길동|    개발|
|     3| 임꺽정|    영업|
|     3| 김철수|    영업|
|     2| 이순신|    연구|
|     4|   null|    기획|
+------+-------+--------+



In [17]:
# name, id
testA = [('A',1),('B',2),('C',3),('D',4)]
# name, myid
testB = [('E',1),('A',2),('C',3),('F',4)]
#spark데이터프레임을 만들고

#inner, left, right, full join 을 테스트하시요

In [18]:
testAA = sqlCtx.createDataFrame( testA, ['name','id'] )
testBB = sqlCtx.createDataFrame( testB,['name', 'myid'] )
testAA.show()
testBB.show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
+----+---+

+----+----+
|name|myid|
+----+----+
|   E|   1|
|   A|   2|
|   C|   3|
|   F|   4|
+----+----+



In [28]:
# inner

testAA.join(testBB, on = ['name']).show()

# right

testAA.join(testBB, on = ['name'], how= 'right').show()

# left

testAA.join(testBB, on = ['name'], how = 'left').show()

# full

full_df = testAA.join(testBB, on = ['name'], how= 'full')

+----+---+----+
|name| id|myid|
+----+---+----+
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+

+----+---+----+
|name| id|myid|
+----+---+----+
|   B|  2|null|
|   D|  4|null|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [29]:
full_df.fillna( {'id':4}).show()

+----+---+----+
|name| id|myid|
+----+---+----+
|   F|  4|   4|
|   E|  4|   1|
|   B|  2|null|
|   D|  4|null|
|   C|  3|   3|
|   A|  1|   2|
+----+---+----+



In [30]:
full_df.dropna(subset=['myid']).show()

+----+----+----+
|name|  id|myid|
+----+----+----+
|   F|null|   4|
|   E|null|   1|
|   C|   3|   3|
|   A|   1|   2|
+----+----+----+



# unionall

- join 말고도, unionall을 통해서 bindrow 할 수도 있다.

In [31]:
testAA.unionAll( testBB).show()

+----+---+
|name| id|
+----+---+
|   A|  1|
|   B|  2|
|   C|  3|
|   D|  4|
|   E|  1|
|   A|  2|
|   C|  3|
|   F|  4|
+----+---+

